Dependencies

In [1]:
%pip install gensim
%pip install nltk

from gensim.models.word2vec import LineSentence
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
import random
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


Pre-processing corpus

In [32]:
def preprocessWord(word):
    word = word.lower().strip()
    
    if word in stop_words:
        return None
    
    word = lemmatizer.lemmatize(word)

    return word

# Load the corpus
corpus = LineSentence('corpus.txt')
docs = [doc for doc in corpus]
tokens = []
for doc in docs:
    tokens.extend(doc)
print(len(tokens))

nltk.download('stopwords')
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

print("Lowercasing...")
tokens = [token.lower() for token in tokens]

print("Removing stopwords...")
tokens = [token for token in tokens if token not in stop_words]

print("Lemmatizing...")
tokens = [lemmatizer.lemmatize(token) for token in tokens]

print("Grouping into sentences...")
sentences = []
sentence = []
counter = 0
words_per_sentence = 10
for token in tokens:
    counter += 1
    if counter > words_per_sentence:
        sentences.append(sentence)
        sentence = [token]
        counter = 1
        continue

    sentence.append(token)

print(len(tokens))
with open('sentences.txt', 'w') as f:
    for sentence in sentences:
        f.write(' '.join(sentence) + '\n')

17005207
Lowercasing...


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/vip/victor.henrique/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Removing stopwords...
Lemmatizing...
Grouping into sentences...
10890638


Filtering out unknown words from evaluation set and sampling it

In [55]:
def getAnalogiesForModel(model):
    final_analogies = []
    with open("questions-words.txt") as f:
        for line in f:
            if line.startswith(":"):
                continue

            line = line.split()
            words = line[:3]
            words = [preprocessWord(word) for word in words if preprocessWord(word) is not None]
            target = preprocessWord(line[3])

            if target not in dummy_model.wv or target is None:
                continue
            
            final_words = []
            for word in words:
                if word in dummy_model.wv:
                    final_words.append(word)

            if len(final_words) == 0:
                continue

            final_analogies.append(final_words + [target])

    # Randomly truncate the analogies
    random.shuffle(final_analogies)
    final_analogies = final_analogies[:500]
    return final_analogies

Defining evaluation procedure

In [56]:
def distanceToMostSimilarWord(input_words: list, true_word: str) -> float:
    similar_words = model.wv.most_similar(input_words)
    most_similar_word = similar_words[0][0]
    return model.wv.distance(true_word, most_similar_word)

def evaluateModel(model):
    analogies = getAnalogiesForModel(model)
    nb_lines = len(analogies)
    avg_distance = 0
    counter = 0
    for analogy in analogies:
        counter += 1
        words = analogy[:-1]
        target = analogy[-1]

        distance = distanceToMostSimilarWord(words, target)
        avg_distance += distance

        job_percentage = 100*(counter/nb_lines)
        # print(f"{job_percentage: .2f}%...", end="\r")

    avg_distance = avg_distance / counter
    # print(f"Average distance: {avg_distance: .2f}")
    return avg_distance

In [ ]:
print("Training model...")
model = Word2Vec(sentences=sentences, vector_size=100, window=5, min_count=1, workers=8)